# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [6]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 116us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 36us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 35

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [9]:
# Train the model 
normalized_input_model.fit(X_train,
                          y_train,
                          batch_size=32,
                          epochs=150,
                          validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 124us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 36us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 35

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

In [12]:
normalized_model.fit(X_train,
                    y_train_scaled,
                    batch_size=32,
                    epochs=150,
                    validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 134us/step - loss: 0.4364 - mean_squared_error: 0.4364 - val_loss: 0.1987 - val_mean_squared_error: 0.1987
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1612 - val_mean_squared_error: 0.1612
Epoch 3/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1589 - val_mean_squared_error: 0.1589
Epoch 4/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.1639 - mean_squared_error: 0.1639 - val_loss: 0.1374 - val_mean_squared_error: 0.1374
Epoch 5/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.1477 - mean_squared_error: 0.1477 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 6/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0318 - mean_squared_error: 0.0318 - val_loss: 0.1013 - val_mean_squared_error: 0.1013
Epoch 51/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 52/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0309 - mean_squared_error: 0.0309 - val_loss: 0.1027 - val_mean_squared_error: 0.1027
Epoch 53/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1031 - val_mean_squared_error: 0.1031
Epoch 54/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1045 - val_mean_squared_error: 0.1045
Epoch 55/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1036 - val_mean_squared_error: 0.103

1051/1051 [==============================] - 0s 33us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1130 - val_mean_squared_error: 0.1130
Epoch 100/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1122 - val_mean_squared_error: 0.1122
Epoch 101/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1135 - val_mean_squared_error: 0.1135
Epoch 102/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1129 - val_mean_squared_error: 0.1129
Epoch 103/150
1051/1051 [==============================] - 0s 34us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1132 - val_mean_squared_error: 0.1132
Epoch 104/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.1140 - val_mean_squared_error: 0.1140
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 149/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 150/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1196 - val_mean_squared_error: 0.1196


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 21us/step


[0.0077693124463893365, 0.0077693124463893365]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 27us/step


[0.11961547662096784, 0.11961547662096784]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

27178.480446302965

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 145us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1699 - val_mean_squared_error: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.2218 - mean_squared_error: 0.2218 - val_loss: 0.1431 - val_mean_squared_error: 0.1431
Epoch 3/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1794 - mean_squared_error: 0.1794 - val_loss: 0.1378 - val_mean_squared_error: 0.1378
Epoch 4/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1573 - mean_squared_error: 0.1573 - val_loss: 0.1249 - val_mean_squared_error: 0.1249
Epoch 5/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1195 - val_mean_squared_error: 0.1195
Epoch 6/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1266 - mean_squared_error: 0.1266 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 51/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0296 - mean_squared_error: 0.0296 - val_loss: 0.1093 - val_mean_squared_error: 0.1093
Epoch 52/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1102 - val_mean_squared_error: 0.1102
Epoch 53/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1105 - val_mean_squared_error: 0.1105
Epoch 54/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 55/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1102 - val_mean_squared_error: 0.110

1051/1051 [==============================] - 0s 37us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 100/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 101/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 102/150
1051/1051 [==============================] - 0s 33us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1181 - val_mean_squared_error: 0.1181
Epoch 103/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 104/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1206 - val_mean_squared_error: 0.1206
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1242 - val_mean_squared_error: 0.1242
Epoch 149/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1238 - val_mean_squared_error: 0.1238
Epoch 150/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1243 - val_mean_squared_error: 0.1243


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 21us/step


[0.00825811535976012, 0.00825811535976012]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 27us/step


[0.12433481277338453, 0.12433481277338453]

## Lecun Initialization 

In the cell below, specify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', kernel_initializer='lecun_normal', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 158us/step - loss: 0.4776 - mean_squared_error: 0.4776 - val_loss: 0.1465 - val_mean_squared_error: 0.1465
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.2097 - mean_squared_error: 0.2097 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 3/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.1699 - mean_squared_error: 0.1699 - val_loss: 0.1169 - val_mean_squared_error: 0.1169
Epoch 4/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 5/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1328 - mean_squared_error: 0.1328 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 6/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1197 - mean_squared_error: 0.1197 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.1164 - val_mean_squared_error: 0.1164
Epoch 51/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1180 - val_mean_squared_error: 0.1180
Epoch 52/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 53/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0252 - mean_squared_error: 0.0252 - val_loss: 0.1196 - val_mean_squared_error: 0.1196
Epoch 54/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1195 - val_mean_squared_error: 0.1195
Epoch 55/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.1193 - val_mean_squared_error: 0.119

1051/1051 [==============================] - 0s 37us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1284 - val_mean_squared_error: 0.1284
Epoch 100/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1281 - val_mean_squared_error: 0.1281
Epoch 101/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1285 - val_mean_squared_error: 0.1285
Epoch 102/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1267 - val_mean_squared_error: 0.1267
Epoch 103/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1277 - val_mean_squared_error: 0.1277
Epoch 104/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1296 - val_mean_squared_error: 0.1296
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1359 - val_mean_squared_error: 0.1359
Epoch 149/150
1051/1051 [==============================] - 0s 34us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1343 - val_mean_squared_error: 0.1343
Epoch 150/150
1051/1051 [==============================] - 0s 34us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1354 - val_mean_squared_error: 0.1354


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 21us/step


[0.007048643891966337, 0.007048643891966337]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 28us/step


[0.13541514798998833, 0.13541514798998833]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                     loss='mse',
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 176us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1370 - val_mean_squared_error: 0.1370
Epoch 2/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1666 - mean_squared_error: 0.1666 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 3/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.1242 - mean_squared_error: 0.1242 - val_loss: 0.1228 - val_mean_squared_error: 0.1228
Epoch 4/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.1091 - val_mean_squared_error: 0.1091
Epoch 5/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0888 - mean_squared_error: 0.0888 - val_loss: 0.1000 - val_mean_squared_error: 0.1000
Epoch 6/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0661 - mean_squared_error: 0.0661 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.1166 - val_mean_squared_error: 0.1166
Epoch 51/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0960 - val_mean_squared_error: 0.0960
Epoch 52/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0912 - val_mean_squared_error: 0.0912
Epoch 53/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0855 - val_mean_squared_error: 0.0855
Epoch 54/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.0874 - val_mean_squared_error: 0.0874
Epoch 55/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0937 - val_mean_squared_error: 0.093

1051/1051 [==============================] - 0s 39us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0875 - val_mean_squared_error: 0.0875
Epoch 100/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0863 - val_mean_squared_error: 0.0863
Epoch 101/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0965 - val_mean_squared_error: 0.0965
Epoch 102/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0906 - val_mean_squared_error: 0.0906
Epoch 103/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0917 - val_mean_squared_error: 0.0917
Epoch 104/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.1010 - val_mean_squared_error: 0.1010
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0861 - val_mean_squared_error: 0.0861
Epoch 149/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0871 - val_mean_squared_error: 0.0871
Epoch 150/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0872 - val_mean_squared_error: 0.0872


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.0025695306209087033, 0.0025695306209087033]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 27us/step


[0.08716014738658535, 0.08716014738658535]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [27]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 357us/step - loss: 0.3727 - mean_squared_error: 0.3727 - val_loss: 0.1650 - val_mean_squared_error: 0.1650
Epoch 2/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1312 - val_mean_squared_error: 0.1312
Epoch 3/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1100 - mean_squared_error: 0.1100 - val_loss: 0.1036 - val_mean_squared_error: 0.1036
Epoch 4/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0856 - mean_squared_error: 0.0856 - val_loss: 0.1026 - val_mean_squared_error: 0.1026
Epoch 5/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0732 - mean_squared_error: 0.0732 - val_loss: 0.1035 - val_mean_squared_error: 0.1035
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0551 - mean_squared_error: 0.0551 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 43us/step - loss: 8.9035e-04 - mean_squared_error: 8.9035e-04 - val_loss: 0.1004 - val_mean_squared_error: 0.1004
Epoch 51/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.1033 - val_mean_squared_error: 0.1033
Epoch 52/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0997 - val_mean_squared_error: 0.0997
Epoch 53/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.1014 - val_mean_squared_error: 0.1014
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.1071 - val_mean_squared_erro

Epoch 99/150
1051/1051 [==============================] - 0s 41us/step - loss: 6.5030e-04 - mean_squared_error: 6.5030e-04 - val_loss: 0.0951 - val_mean_squared_error: 0.0951
Epoch 100/150
1051/1051 [==============================] - 0s 40us/step - loss: 5.5846e-04 - mean_squared_error: 5.5846e-04 - val_loss: 0.0926 - val_mean_squared_error: 0.0926
Epoch 101/150
1051/1051 [==============================] - 0s 39us/step - loss: 6.1111e-04 - mean_squared_error: 6.1111e-04 - val_loss: 0.0952 - val_mean_squared_error: 0.0952
Epoch 102/150
1051/1051 [==============================] - 0s 41us/step - loss: 5.7706e-04 - mean_squared_error: 5.7706e-04 - val_loss: 0.0937 - val_mean_squared_error: 0.0937
Epoch 103/150
1051/1051 [==============================] - 0s 39us/step - loss: 5.6679e-04 - mean_squared_error: 5.6679e-04 - val_loss: 0.0948 - val_mean_squared_error: 0.0948
Epoch 104/150
1051/1051 [==============================] - 0s 40us/step - loss: 5.5248e-04 - mean_squared_error: 5.5248e-

Epoch 147/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0876 - val_mean_squared_error: 0.0876
Epoch 148/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0837 - val_mean_squared_error: 0.0837
Epoch 149/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0907 - val_mean_squared_error: 0.0907
Epoch 150/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0885 - val_mean_squared_error: 0.0885


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.003585413960539705, 0.003585413960539705]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 28us/step


[0.08845674262130668, 0.08845674262130668]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 32us/step


[0.16909882793687794, 0.16909882793687794]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [32]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# RMSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

32314.825077497197

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.